# ReLU (Rectified Linear Unit) — Key Notes

## Definition
The **Rectified Linear Unit (ReLU)** is one of the most widely used activation functions in modern neural networks.  
It is defined as:

$$f(x) = \max(0, x)$$

---

## Intuition
- If the input is **positive**, ReLU just passes it through (acts like identity).
- If the input is **negative**, ReLU outputs **0** (it "rectifies" it).

---

## Derivative (for backpropagation)
ReLU is simple to differentiate:

$$f'(x) = \begin{cases}
1 & \text{if } x > 0 \\
0 & \text{if } x \leq 0
\end{cases}$$

- For positive values → gradient = 1 (no shrinking like sigmoid/tanh).
- For negative values → gradient = 0 (inactive neuron).

---

## Why ReLU is Popular
1. **Computationally cheap**  
   - Just compares with 0, no exponentials or divisions.
   
2. **Avoids vanishing gradients** (mostly)  
   - Unlike sigmoid/tanh, the gradient for positive inputs is 1, so deep networks can propagate gradients better.

3. **Sparsity**  
   - Many neurons output exactly 0 → network becomes sparse, which can help generalization and efficiency.

4. **Linear behavior for positive inputs**  
   - Makes optimization easier compared to bounded activations.

---

## Drawbacks
- **Dying ReLU problem**: Neurons that fall into the negative side can get stuck (always output 0, never update).
- Solutions: Leaky ReLU, Parametric ReLU, ELU, GELU, etc.

---

## Mathematical Properties

The ReLU function has these key mathematical characteristics:

**Function**: 
$$f(x) = \max(0, x) = \begin{cases}
x & \text{if } x \geq 0 \\
0 & \text{if } x < 0
\end{cases}$$

**Derivative**: 
$$f'(x) = \begin{cases}
1 & \text{if } x > 0 \\
0 & \text{if } x \leq 0
\end{cases}$$

**Note**: The derivative at $x = 0$ is technically undefined, but in practice we often set $f'(0) = 0$.

---

## Summary
- **Formula**: $f(x) = \max(0, x)$  
- **Derivative**: $f'(x) = 1$ (if $x > 0$), else $0$  
- **Good for**: Deep networks, faster training, avoiding vanishing gradients  
- **Risk**: Some neurons can "die"  


In [ ]:
## Visualization

```python
import numpy as np
import matplotlib.pyplot as plt

# Create input values
x = np.linspace(-5, 5, 100)

# Apply ReLU function: f(x) = max(0, x)
relu_output = np.maximum(0, x)

# Plot the function
plt.figure(figsize=(10, 6))

# Plot ReLU function
plt.subplot(1, 2, 1)
plt.plot(x, relu_output, 'b-', linewidth=2, label='ReLU: f(x) = max(0, x)')
plt.axhline(y=0, color='k', linestyle='--', alpha=0.3)
plt.axvline(x=0, color='k', linestyle='--', alpha=0.3)
plt.grid(True, alpha=0.3)
plt.xlabel('Input (x)')
plt.ylabel('Output f(x)')
plt.title('ReLU Activation Function')
plt.legend()

# Plot ReLU derivative
plt.subplot(1, 2, 2)
relu_derivative = np.where(x > 0, 1, 0)
plt.plot(x, relu_derivative, 'r-', linewidth=2, label="ReLU': f'(x)")
plt.axhline(y=0, color='k', linestyle='--', alpha=0.3)
plt.axvline(x=0, color='k', linestyle='--', alpha=0.3)
plt.grid(True, alpha=0.3)
plt.xlabel('Input (x)')
plt.ylabel("Derivative f'(x)")
plt.title('ReLU Derivative')
plt.legend()
plt.ylim(-0.1, 1.1)

plt.tight_layout()
plt.show()

# Demonstrate the mathematical properties
print("ReLU Function Examples:")
test_values = [-2, -1, 0, 1, 2]
for val in test_values:
    result = max(0, val)
    derivative = 1 if val > 0 else 0
    print(f"f({val:2}) = max(0, {val:2}) = {result:2}, f'({val:2}) = {derivative}")
```
